In [2]:
from matplotlib import pyplot as plt 
import numpy as np 
import pandas as pd 
import plotly.express as px
import plotly.io as pio
import plotly.graph_objs as go
import plotly.offline as pyo
import pickle

from collections import Counter

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

from matplotlib import rc

### Load pickle

In [3]:
# NEW FILE
read_input = open('data/EDADS_subtype_timelines_agecorrected_opt.pickle','rb')
load_inputs = pickle.load(read_input)
read_input.close()

T, S, Sboot = load_inputs

diagnosis = np.load('data/diagnosis.npy', allow_pickle=True)

/Users/macos/opt/anaconda3/envs/umc/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning:

Trying to unpickle estimator MinCovDet from version 0.24.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations



- T --> timeline (timeline object)

- S --> subjects (dict)

- Sboot --> subjects bootstrappng

In [4]:
data = pd.read_csv("data/EDADS_data.csv")
data

,PTID,Diagnosis,Temporal_lobe,Superior_frontal_gyrus,Middle_frontal_gyrus,Inferior_frontal_gyrus,Gyrus_rectus,Orbitofrontal_gyri,Precentral_gyrus,Postcentral_gyrus,...,Insula,Gyrus_cinguli_anterior_part,Gyrus_cinguli_posterior_part,Parahippocampal_and_ambient_gyri,Thalamus,Caudate,Putamen,Hippocampus,Amygdala,Accumbens-area
0,7,AD,83566.056692,33020.148732,28540.538397,18527.609787,10157.464125,12473.242003,30172.977024,15764.342697,...,11857.677956,6926.280007,7815.323434,3122.949573,12328.973225,5415.988765,7695.809731,7474.906168,2430.773513,753.744471
1,22,AD,83412.284925,39859.984143,39030.182207,19510.889331,9808.808042,13533.166716,28546.053343,15816.434199,...,12270.341140,6477.125298,9444.821944,2989.194018,12944.300524,6408.795543,7527.756561,6204.957352,2324.141511,731.965796
2,102,MCI,87197.458113,39277.350454,37723.799331,19528.835525,9518.220914,13864.818489,27569.909012,16975.480266,...,11518.941570,8368.484444,9526.388301,4183.742608,12058.404451,7039.811314,9171.199629,7431.517579,2506.464608,852.351538
3,104,AD,93552.949510,39517.631037,39496.626351,19645.405918,9964.159946,14450.284333,32569.843916,21382.004244,...,12587.462573,9099.113868,10912.146364,4141.349410,13775.516130,6612.811008,8811.961234,7861.127965,3247.165795,937.634259
4,128,AD,76417.884687,32941.405709,28601.983351,15643.148705,8689.951457,11534.100083,27396.356566,15690.089238,...,12229.451522,5922.568049,5684.362430,2993.957543,13003.261663,7167.305184,8048.781423,7812.890552,3372.451093,870.288043
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2231,11450,SCD,111723.449181,49370.777062,39717.356744,21849.682495,11342.711895,15246.257259,36896.331182,21209.463637,...,15582.871801,7414.078302,9997.571206,4187.574210,13038.777596,6491.267001,10143.022664,8184.380340,3292.786216,946.811105
2232,11464,AD,89362.233983,39523.138439,33926.936849,19723.548155,9336.994087,13439.694879,29007.111647,16917.344367,...,13048.604766,7717.104028,8814.609043,3937.162293,13031.619001,7131.479707,8040.804354,6382.778078,2694.860552,1012.276222
2233,11468,AD,90075.762244,40445.360705,38497.805450,18633.197249,10037.258608,14414.538390,31920.963294,18116.426704,...,13287.132095,6649.841997,10543.124485,4246.387605,13196.626444,7565.194490,8586.070267,6697.416152,2951.714963,669.053043
2234,11469,SCD,104889.475458,40588.737627,39762.753958,21773.495827,10025.559598,13046.737370,31137.800395,17834.311789,...,15014.550890,7996.687543,8367.250970,3451.949857,15541.447190,8766.421899,10536.447666,8152.324085,3200.446625,1122.250589


In [7]:
len(Sboot[0]['staging'])

2236

In [13]:
data['PTID']

0           7
1          22
2         102
3         104
4         128
        ...  
2231    11450
2232    11464
2233    11468
2234    11469
2235    11482
Name: PTID, Length: 2236, dtype: int64

In [18]:
Sboot[0]['staging'][data['PTID']==7]

array([0.7349252])

## Staging - boxplots

In [38]:
def individual_staging(data, Sboot, patient_id, color='#000000', fontsize=18, width=900, height=400):
    
    """
    Creates a boxplot
    :param Sboot: bootstrap samples for individual patients
    :param patient_id: patient id
    :param color: hex color values (optional)
    :param width: int (optional)
    :param height: int (optional)
    :return: plotly go Box figure
    """
        
    boot = []
    
    for b in range(len(Sboot)):
        boot.append(Sboot[b]['staging'][data['PTID']==patient_id][0])
        
    print(boot)
        
    fig = go.Figure()

    fig.add_trace(go.Box(x=boot, 
                         name='',
                         fillcolor=color,
                         line_color='#000000',
                         opacity=0.8))
    
    # fig.update_xaxes(range=[-0.05, 1.0])

    fig.update_layout(
                xaxis_title="Disease Stage",
                showlegend=False,
                autosize = False,
                width=width,
                height=height
            )
    
    fig.update_xaxes(title_font_size=fontsize)
    
    return fig

In [39]:
patient_id = 7
color = '#4daf4a'
width=900
height=400
fontsize=30

In [40]:
p = individual_staging(data=data,
                       Sboot=Sboot,
                      patient_id=patient_id,
                      color=color,
                       width=width,
                       height=height,
                       fontsize=fontsize
                      )

p

[0.7349251980252052, 0.6628281670851455, 0.7327947363593039, 0.721149751184964, 0.7407285239647644, 0.7543211639488184, 0.6858201595625015, 0.7416311497091577, 0.731796766889404, 0.7227500964981498, 0.7161836277465901, 0.7259401821664032, 0.7065042835843048, 0.7320257259945284, 0.7312376528542953, 0.7194821530847372, 0.7174594705617873, 0.6910278180221237, 0.6987194203910563, 0.7332326461837005, 0.7022495804494648, 0.6948490958808962, 0.6945867573527158, 0.7263327109515002, 0.6977224402501215, 0.7252764955818317, 0.7361030291715684, 0.7099064715259344, 0.7313797619849044, 0.7469654344715336, 0.7006043398353198, 0.7317574261936489, 0.7203436104684732, 0.7463456180831441, 0.7159870275679284, 0.6889516860653592, 0.7267684909780432, 0.7162221504305273, 0.7768561206705519, 0.7225937969363608, 0.739575793324219, 0.7128253874328654, 0.6805699660741756, 0.7767254132135448, 0.6992564672911435, 0.6925413211511955, 0.7165750373505475, 0.7347077414955141, 0.7028984773968858, 0.7113053285004991, 0.